In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
import pywt.data
import torch
original = torch.tensor([[1.,2.],[1.,1.]])
coeffs2 = pywt.dwt2(original, 'haar')
coeffs2
#haar小波变换的高通滤波器h={pow(1/2,1/2),pow(1/2,1/2)};低通滤波器h={pow(1/2,1/2),-pow(1/2,1/2)}

(array([[2.4999998]], dtype=float32),
 (array([[0.49999997]], dtype=float32),
  array([[-0.49999994]], dtype=float32),
  array([[-0.49999997]], dtype=float32)))

In [4]:
#矩阵乘法实现卷积
import torch

def unfold_matrix(x,w):
    n,m = x.shape[0:2]
    k,l = w.shape[0:2]
    xx = torch.zeros(((n-k+1)*(m-l+1),k*l))
    row_num = 0
    for i in range(n-k+1):
        for j in range(m-l+1):
            xx[row_num,:] = x[i:i+k, j:j+l].flatten()
            row_num = row_num + 1
    return xx

def conv_matrix_tensor(x, w):
    n, m = x.shape[0:]
    k, l = w.shape[0:]
    x_unfolded = unfold_matrix(x, w)
    w_flat = w.flatten()#flatten()--把矩阵展开，变为向量
    yy = torch.matmul(x_unfolded, w_flat)#expected scalar type Float
    yy = yy.reshape((n-k+1, m-l+1))
    return yy

In [9]:
def haar_d2wt(x):
    
    #滤波器，h——高通滤波器，g——低通滤波器
    a = pow(1/2,1/2)
    h = torch.Tensor([[a, a]]).T
    g = torch.Tensor([[a, -a]]).T
    #给矩阵加一圈零，使得之后符合变换运算过程
    _x_ = torch.zeros(x.shape[0],1)                   
    x = torch.cat((_x_, x, _x_ ),1)
    __x__ = torch.zeros(1,x.shape[1])
    x = torch.cat((__x__, x, __x__),0)
    #矩阵乘法实现卷积取偶数行
    L = conv_matrix_tensor(x, h)
    H = conv_matrix_tensor(x, g)
    LL = conv_matrix_tensor(L, h.T)[1::2,1::2]
    LH = conv_matrix_tensor(L, g.T)[1::2,1::2]
    HL = conv_matrix_tensor(H, h.T)[1::2,1::2]
    HH = conv_matrix_tensor(H, g.T)[1::2,1::2]
    
    return LL, LH, HL, HH

x = torch.tensor([[1., 2.,3., 1., 2.,3. ],[4., 5., 3., 1., 2.,3.],[6., 7., 1., 1., 2.,3. ], [5., 9., 6.,1., 2.,3. ]])
LL, LH, HL, HH = haar_d2wt(x)
print(LL, '\n', LH, '\n', HL, '\n', HH)

tensor([[ 6.0000,  4.0000,  5.0000],
        [13.5000,  4.5000,  5.0000]]) 
 tensor([[-1.0000,  2.0000, -1.0000],
        [-2.5000,  2.5000, -1.0000]]) 
 tensor([[-3.0000,  0.0000,  0.0000],
        [-0.5000, -2.5000,  0.0000]]) 
 tensor([[ 0.0000,  0.0000,  0.0000],
        [ 1.5000, -2.5000,  0.0000]])


In [10]:
coeffs2 = pywt.dwt2(x, 'haar')
coeffs2

(array([[ 6.       ,  3.9999998,  4.9999995],
        [13.5      ,  4.4999995,  4.9999995]], dtype=float32),
 (array([[-2.9999998 ,  0.        ,  0.        ],
         [-0.49999985, -2.4999998 ,  0.        ]], dtype=float32),
  array([[-1.       ,  1.9999998, -0.9999999],
         [-2.4999995,  2.4999998, -0.9999999]], dtype=float32),
  array([[ 0.       ,  0.       ,  0.       ],
         [ 1.4999995, -2.4999998,  0.       ]], dtype=float32)))

In [14]:
x

tensor([[1., 2., 1., 2.],
        [2., 3., 1., 2.],
        [2., 3., 1., 2.],
        [2., 3., 1., 2.]])

In [1]:
from Pytorch_wavelets import DWTForward
xfm = DWTForward(2, 'zero', wave=0.3)

ModuleNotFoundError: No module named 'Pytorch_wavelets'

In [ ]:
student_l, student_h = xfm(fake_B)
teacher_l, teacher_h = xfm(real_B)

In [ ]:
for index in range(len(student_h)):
    self.loss_kd += torch.nn.functional.mse_loss(teacher_h[index],student_h[index]) * 0.2